# QNotebook Test - Verifica Funzionalità

Questo notebook testa le principali funzionalità del plugin QNotebook per QGIS.

In [ ]:
# Test 1: Import base e verifica ambiente QGIS
print("Test 1: Verifica ambiente QGIS")
print("="*50)

# Verifica che siamo in QGIS
try:
    print(f"QGIS Version: {Qgis.version()}")
    print(f"QGIS Version Int: {Qgis.versionInt()}")
    print(f"Interface disponibile: {iface is not None}")
    print(f"Project instance: {QgsProject.instance() is not None}")
    print("✓ Ambiente QGIS OK")
except Exception as e:
    print(f"✗ Errore: {e}")

In [ ]:
# Test 2: Accesso al canvas e al progetto
print("\nTest 2: Canvas e Progetto")
print("="*50)

canvas = iface.mapCanvas()
project = QgsProject.instance()

print(f"Canvas scale: 1:{canvas.scale():.0f}")
print(f"CRS: {canvas.mapSettings().destinationCrs().authid()}")
print(f"Extent: {canvas.extent().toString(2)}")
print(f"Numero di layer nel progetto: {len(project.mapLayers())}")

# Lista dei layer
if project.mapLayers():
    print("\nLayer presenti:")
    for name, layer in project.mapLayers().items():
        print(f"  - {layer.name()} ({layer.type()})")
else:
    print("\nNessun layer caricato nel progetto")

In [ ]:
# Test 3: Creazione di un layer di test
print("\nTest 3: Creazione Layer di Test")
print("="*50)

from qgis.core import QgsVectorLayer, QgsFeature, QgsGeometry, QgsPointXY, QgsField
from qgis.PyQt.QtCore import QVariant
import random

# Crea un layer di punti in memoria
layer = QgsVectorLayer("Point?crs=EPSG:4326", "Test Points", "memory")
provider = layer.dataProvider()

# Aggiungi campi
provider.addAttributes([
    QgsField("id", QVariant.Int),
    QgsField("name", QVariant.String),
    QgsField("value", QVariant.Double)
])
layer.updateFields()

# Aggiungi 10 punti casuali
features = []
for i in range(10):
    feat = QgsFeature()
    # Coordinate casuali (Roma area)
    x = 12.4 + random.random() * 0.2
    y = 41.8 + random.random() * 0.2
    feat.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(x, y)))
    feat.setAttributes([i, f"Point_{i}", random.random() * 100])
    features.append(feat)

provider.addFeatures(features)
layer.updateExtents()

# Aggiungi al progetto
QgsProject.instance().addMapLayer(layer)

print(f"✓ Layer '{layer.name()}' creato con {layer.featureCount()} features")
print(f"  Extent: {layer.extent().toString(2)}")
print(f"  Fields: {[field.name() for field in layer.fields()]}")

In [ ]:
# Test 4: Analisi dati del layer
print("\nTest 4: Analisi Dati")
print("="*50)

if layer:
    # Estrai valori
    values = []
    names = []
    
    for feature in layer.getFeatures():
        values.append(feature['value'])
        names.append(feature['name'])
    
    # Statistiche
    import statistics
    
    print(f"Numero features: {len(values)}")
    print(f"Valore minimo: {min(values):.2f}")
    print(f"Valore massimo: {max(values):.2f}")
    print(f"Media: {statistics.mean(values):.2f}")
    print(f"Mediana: {statistics.median(values):.2f}")
    print(f"Dev. Standard: {statistics.stdev(values):.2f}")
    
    print("\nPrime 5 features:")
    for i, (name, val) in enumerate(zip(names[:5], values[:5])):
        print(f"  {name}: {val:.2f}")

In [ ]:
# Test 5: Visualizzazione con matplotlib
print("\nTest 5: Visualizzazione Matplotlib")
print("="*50)

try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    if layer and values:
        # Crea figura con subplot
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # Istogramma
        axes[0].hist(values, bins=5, edgecolor='black', alpha=0.7, color='skyblue')
        axes[0].set_title('Distribuzione Valori')
        axes[0].set_xlabel('Valore')
        axes[0].set_ylabel('Frequenza')
        axes[0].grid(True, alpha=0.3)
        
        # Scatter plot con indice
        axes[1].scatter(range(len(values)), values, alpha=0.6, s=100, c=values, cmap='viridis')
        axes[1].set_title('Valori per Feature')
        axes[1].set_xlabel('Feature Index')
        axes[1].set_ylabel('Valore')
        axes[1].grid(True, alpha=0.3)
        
        # Aggiungi colorbar
        cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
        cbar.set_label('Valore')
        
        plt.suptitle(f'Analisi Layer: {layer.name()}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("✓ Grafici generati con successo")
    else:
        print("✗ Nessun dato da visualizzare")
        
except ImportError:
    print("✗ Matplotlib non disponibile")
except Exception as e:
    print(f"✗ Errore: {e}")

In [ ]:
# Test 6: Processing
print("\nTest 6: QGIS Processing")
print("="*50)

try:
    import processing
    
    if layer:
        # Buffer sui punti
        result = processing.run("native:buffer", {
            'INPUT': layer,
            'DISTANCE': 0.01,  # 0.01 gradi
            'SEGMENTS': 8,
            'OUTPUT': 'memory:'
        })
        
        buffer_layer = result['OUTPUT']
        buffer_layer.setName("Test Buffers")
        
        # Aggiungi al progetto
        QgsProject.instance().addMapLayer(buffer_layer)
        
        print(f"✓ Buffer creato: {buffer_layer.featureCount()} features")
        print(f"  Area totale: {sum([f.geometry().area() for f in buffer_layer.getFeatures()]):.6f} gradi²")
        
        # Converti in centroidi
        result2 = processing.run("native:centroids", {
            'INPUT': buffer_layer,
            'OUTPUT': 'memory:'
        })
        
        centroid_layer = result2['OUTPUT']
        print(f"✓ Centroidi calcolati: {centroid_layer.featureCount()} punti")
        
    else:
        print("✗ Nessun layer disponibile per il processing")
        
except Exception as e:
    print(f"✗ Errore nel processing: {e}")

In [ ]:
# Test 7: Selezione e zoom
print("\nTest 7: Interazione con Canvas")
print("="*50)

if layer:
    # Seleziona le prime 3 features
    layer.selectByIds([0, 1, 2])
    print(f"✓ Selezionate {layer.selectedFeatureCount()} features")
    
    # Zoom all'extent del layer
    canvas.setExtent(layer.extent())
    canvas.refresh()
    print(f"✓ Zoom eseguito all'extent del layer")
    
    # Info sulla selezione
    for feat in layer.selectedFeatures():
        geom = feat.geometry()
        point = geom.asPoint()
        print(f"  Feature {feat['id']}: ({point.x():.4f}, {point.y():.4f})")

In [ ]:
# Test 8: Stile del layer
print("\nTest 8: Stile e Simbologia")
print("="*50)

from qgis.core import QgsSymbol, QgsSimpleMarkerSymbolLayer, QgsGraduatedSymbolRenderer
from qgis.PyQt.QtGui import QColor

if layer:
    # Crea simbolo personalizzato
    symbol = QgsSymbol.defaultSymbol(layer.geometryType())
    symbol.setColor(QColor(255, 0, 0))
    symbol.setSize(5)
    
    # Renderer graduato basato sul campo 'value'
    renderer = QgsGraduatedSymbolRenderer('value')
    renderer.setClassAttribute('value')
    
    # Crea classi graduate
    method = QgsGraduatedSymbolRenderer.EqualInterval
    renderer.updateClasses(layer, method, 3)
    
    # Colori graduati
    colors = [QColor(255, 255, 178), QColor(254, 178, 76), QColor(240, 59, 32)]
    for i, range_item in enumerate(renderer.ranges()):
        if i < len(colors):
            range_item.symbol().setColor(colors[i])
    
    layer.setRenderer(renderer)
    layer.triggerRepaint()
    
    print("✓ Stile graduato applicato")
    print(f"  Numero di classi: {len(renderer.ranges())}")
    for i, r in enumerate(renderer.ranges()):
        print(f"  Classe {i+1}: {r.lowerValue():.2f} - {r.upperValue():.2f}")

In [ ]:
# Test 9: Salvataggio risultati
print("\nTest 9: Export Dati")
print("="*50)

import os
import tempfile

if layer:
    # Directory temporanea
    temp_dir = tempfile.gettempdir()
    
    # Salva come GeoJSON
    geojson_path = os.path.join(temp_dir, "test_points.geojson")
    options = QgsVectorFileWriter.SaveVectorOptions()
    options.driverName = "GeoJSON"
    
    error = QgsVectorFileWriter.writeAsVectorFormatV2(
        layer,
        geojson_path,
        QgsCoordinateTransformContext(),
        options
    )
    
    if error[0] == QgsVectorFileWriter.NoError:
        print(f"✓ Layer salvato come GeoJSON")
        print(f"  Path: {geojson_path}")
        print(f"  Size: {os.path.getsize(geojson_path)} bytes")
    else:
        print(f"✗ Errore nel salvataggio: {error[1]}")

## Test Completato!

Se tutti i test sono stati eseguiti con successo, il plugin QNotebook funziona correttamente.

### Funzionalità testate:
- ✓ Ambiente QGIS e accesso all'interfaccia
- ✓ Creazione e manipolazione layer
- ✓ Analisi dati
- ✓ Visualizzazioni con matplotlib
- ✓ Processing tools
- ✓ Interazione con il canvas
- ✓ Stili e simbologia
- ✓ Export dati